In [1]:
pip install h3

  Using cached h3-3.7.6-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.1 MB)
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install keplergl

Note: you may need to restart the kernel to use updated packages.


In [10]:
import h3 as h3
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from scipy import stats
from pyhive import presto
from keplergl import KeplerGl

import warnings
warnings.filterwarnings('ignore')

In [4]:
from datetime import datetime, timedelta


In [22]:
# Step 2: Generate date range
start_date = datetime(2023, 7, 10)  # Replace with your desired start date
end_date = datetime(2023, 7, 11)   # Replace with your desired end date
date_range = [start_date + timedelta(days=x) for x in range((end_date - start_date).days + 1)]


In [11]:
## Connection
connection = presto.connect(
        host='presto-gateway.serving.data.production.internal',
        port=80,
        protocol='http',
        catalog='hive',
        username='manoj.ravirajan@rapido.bike'
)

In [23]:
# Step 3: Iterate over the date range and execute SQL queries
raw_fare_estimates_enriched = []
for date in date_range:
    formatted_date = date.strftime('%Y%m%d')
    query = f""" SELECT distinct yyyymmdd, order_id FROM orders.order_logs_snapshot WHERE yyyymmdd = '{formatted_date}' AND order_status != 'dropped' and city_name = 'Chennai' """  # Replace your_table and date_column with actual values
    df = pd.read_sql(query, connection)
    raw_fare_estimates_enriched.append(df)

# Concatenate all the results into a single DataFrame
final_result = pd.concat(raw_fare_estimates_enriched)

In [24]:
final_result

,yyyymmdd,order_id
0,20230710,64ab1458f1c985266fba2432
1,20230710,64ab96efe7c508754f830479
2,20230710,64abf91d1e9978677e56d3c3
3,20230710,64abf6dac5ccb3788e1f5a4e
4,20230710,64ac033bf1c985266fbb9f09
...,...,...
87718,20230711,64ad1603b32571163e047692
87719,20230711,64ad66ef84339b26184d3a6c
87720,20230711,64ad47ff197c7c4d82b69b59
87721,20230711,64ac57a734a7755c65ce3216


In [25]:
final_result.yyyymmdd.describe()

count       225308
unique           2
top       20230710
freq        137585
Name: yyyymmdd, dtype: object

In [ ]:
final_result